In [23]:
import pandas as pd
import sqlite3

# Load orders and users
orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")

# Create SQLite connection
conn = sqlite3.connect(":memory:")

# Load SQL file
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

# Execute SQL script
conn.executescript(sql_script)

# Load restaurants table
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# LEFT JOIN orders with users
merged1 = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

# LEFT JOIN with restaurants
final_df = pd.merge(
    merged1,
    restaurants,
    on="restaurant_id",
    how="left"
)

# Save final dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("✅ Final dataset created: final_food_delivery_dataset.csv")
print(final_df.head())

✅ Final dataset created: final_food_delivery_dataset.csv
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restauran

In [4]:
# read final csv
df = pd.read_csv("final_food_delivery_dataset.csv")
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [88]:
# Which city has the highest total revenue (total_amount) from Gold members?  
gold_df = df[df["membership"] == "Gold"]

result = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by="total_amount", ascending=False)
)

print(result.head())

        city  total_amount
1    Chennai    1080909.79
3       Pune    1003012.32
0  Bangalore     994702.59
2  Hyderabad     896740.19


In [89]:
# Which cuisine has the highest average order value across all orders?
result = (
    df.groupby("cuisine")["total_amount"]
      .mean()
      .reset_index()
      .sort_values(by="total_amount", ascending=False)
)

print(result.head())


   cuisine  total_amount
3  Mexican    808.021344
2  Italian    799.448578
1   Indian    798.466011
0  Chinese    798.389020


In [90]:
# How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
user_spending = (
    df.groupby("user_id")["total_amount"]
      .sum()
      .reset_index()
)

result = user_spending[user_spending["total_amount"] > 1000]

print("Number of users who spent more than ₹1000:", result.shape[0])


Number of users who spent more than ₹1000: 2544


In [91]:
# Which restaurant rating range generated the highest total revenue?
# Create rating ranges
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 - 5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

result = (
    df.groupby("rating_range")["total_amount"]
      .sum()
      .reset_index()
      .sort_values(by="total_amount", ascending=False)
)

print(result)


  rating_range  total_amount
3    4.6 - 5.0    2197030.75
0    3.0 – 3.5    2136772.70
2    4.1 – 4.5    1960326.26
1    3.6 – 4.0    1717494.41


C:\Users\admin\AppData\Local\Temp\ipykernel_17364\3156204521.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


In [92]:
# Among Gold members, which city has the highest average order value?
gold_df = df[df["membership"] == "Gold"]

result = (
    gold_df.groupby("city")["total_amount"]
           .mean()
           .reset_index()
           .sort_values(by="total_amount", ascending=False)
)

print(result.head())

        city  total_amount
1    Chennai    808.459080
2  Hyderabad    806.421034
0  Bangalore    793.223756
3       Pune    781.162243


In [93]:
# Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
# Filter is not needed here, we use all data
result = (
    orders.merge(restaurants[['restaurant_id', 'cuisine']], on='restaurant_id', how='left')
          .groupby("cuisine")["total_amount"]
          .sum()  # total revenue per cuisine
          .reset_index()
          .merge(
              restaurants.groupby("cuisine")["restaurant_id"]
                         .nunique()
                         .reset_index()
                         .rename(columns={"restaurant_id": "num_restaurants"}),
              on="cuisine"
          )
          .sort_values(by=["num_restaurants", "total_amount"], ascending=[True, False])
)

print(result.head(5))


   cuisine  total_amount  num_restaurants
0  Chinese    1930504.65              120
2  Italian    2024203.80              126
1   Indian    1971412.58              126
3  Mexican    2085503.09              128


In [94]:
# What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
# Filter Gold members
gold_df = df[df["membership"] == "Gold"]

# Calculate percentage of total orders
gold_order_percentage = round((len(gold_df) / len(df)) * 100)

print(f"Percentage of orders by Gold members: {gold_order_percentage}%")


Percentage of orders by Gold members: 50%


In [95]:
# Which restaurant has the highest average order value but less than 20 total orders?
result = (
    df[df["restaurant_name_x"].isin([
        "Grand Cafe Punjabi",
        "Grand Restaurant South Indian",
        "Ruchi Mess Multicuisine",
        "Ruchi Foods Chinese"
    ])]
    .groupby("restaurant_name_x")
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
    .query("total_orders < 20")
    .sort_values(by="avg_order_value", ascending=False)
    .head(1)
)

result

,avg_order_value,total_orders
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


In [96]:
# Which combination contributes the highest revenue?
result = (
    df[df["membership"].isin(["Gold", "Regular"]) &
       df["cuisine"].isin(["Indian", "Italian", "Chinese"])]
    .groupby(["membership", "cuisine"])
    .agg(total_revenue=("total_amount", "sum"))
    .reset_index()
    .query(
        "(membership == 'Gold' and cuisine == 'Indian') or "
        "(membership == 'Gold' and cuisine == 'Italian') or "
        "(membership == 'Regular' and cuisine == 'Indian') or "
        "(membership == 'Regular' and cuisine == 'Chinese')"
    )
    .sort_values(by="total_revenue", ascending=False)
    .head(1)
)

result

,membership,cuisine,total_revenue
2,Gold,Italian,1005779.05


In [97]:
# During which quarter of the year is the total revenue highest?
result = (
    df.assign(
        quarter=pd.to_datetime(df["order_date"], dayfirst=True).dt.quarter
    )
    .groupby("quarter")
    .agg(total_revenue=("total_amount", "sum"))
    .reset_index()
    .sort_values(by="total_revenue", ascending=False)
    .head(1)
)

result

,quarter,total_revenue
2,3,2037385.1


In [98]:
# How many total orders were placed by users with Gold membership?
result = (
    df[df["membership"] == "Gold"]
    .agg(total_orders=("order_id", "count"))
)

result

,order_id
total_orders,4987


In [99]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
result = (
    df[df["city"] == "Hyderabad"]
    .agg(total_revenue=("total_amount", "sum"))
    .round(0)
)

result

,total_amount
total_revenue,1889367.0


In [100]:
# How many distinct users placed at least one order?
result = df["user_id"].nunique()
result

2883

In [101]:
# What is the average order value (rounded to 2 decimals) for Gold members?
result = (
    df[df["membership"] == "Gold"]
    .agg(avg_order_value=("total_amount", "mean"))
    .round(2)
)

result

,total_amount
avg_order_value,797.15


In [102]:
# How many orders were placed for restaurants with rating ≥ 4.5?

high_rating_count = len(df[df['rating'] >= 4.5])

print(f"Number of orders with rating >= 4.5: {high_rating_count}")

Number of orders with rating >= 4.5: 3374


In [103]:
# How many orders were placed in the top revenue city among Gold members only?

result = len(df[(df["membership"] == "Gold") & (df["city"] == "Bangalore")])
result

1254

In [87]:
The total number of rows in the final merged dataset is __________.
df.shape

(10000, 12)